<a href="https://colab.research.google.com/github/nikhilbansal23/WebsiteKeywordsDetection/blob/main/Keyword_Detection_on_Websites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
pip install beautifulsoup4

In [14]:
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
train = pd.read_csv('/content/drive/MyDrive/Datasets/Keyword Detection on Websites/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Datasets/Keyword Detection on Websites/test.csv')
tumor = pd.read_csv('/content/drive/MyDrive/Datasets/Keyword Detection on Websites/keyword2tumor_type.csv')

In [5]:
train.head()

,url,doc_id,label
0,http://elbe-elster-klinikum.de/fachbereiche/ch...,1,1
1,http://klinikum-bayreuth.de/einrichtungen/zent...,3,3
2,http://klinikum-braunschweig.de/info.php/?id_o...,4,1
3,http://klinikum-braunschweig.de/info.php/?id_o...,5,1
4,http://klinikum-braunschweig.de/zuweiser/tumor...,6,3


In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extract specific tags or text content based on HTML structure
        # For example:
        # text_content = soup.find('div', {'class': 'content'}).get_text()
        # return text_content
        return soup.get_text()  # This extracts all text content
    except Exception as e:
        print(f"Error fetching content from {url}: {e}")
        return None

# Apply the function to fetch text content for each URL in the DataFrame
train['text_content'] = train['url'].apply(extract_text)



Error fetching content from http://rendsburg.imland.de/de/thoraxchirurgie_kooperati/?f=1: HTTPConnectionPool(host='rendsburg.imland.de', port=80): Max retries exceeded with url: /de/thoraxchirurgie_kooperati/?f=1 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x793bd42e11e0>: Failed to resolve 'rendsburg.imland.de' ([Errno -2] Name or service not known)"))
Error fetching content from http://www.dasdiak.de/bildungsangebote/untermenue/vorsprung-durch-wissen-fort-und-weiterbildung/medizinische-kliniken-und-institute/details.html/?chash=8f2bd33e874f84770d633c55ade760ad&item=67: HTTPConnectionPool(host='www.dasdiak.de', port=80): Max retries exceeded with url: /bildungsangebote/untermenue/vorsprung-durch-wissen-fort-und-weiterbildung/medizinische-kliniken-und-institute/details.html/?chash=8f2bd33e874f84770d633c55ade760ad&item=67 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x793bd4572e60>, 'Connection to www.dasdiak.de timed out. 

In [32]:
df = train

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           100 non-null    object
 1   doc_id        100 non-null    int64 
 2   label         100 non-null    int64 
 3   clean_text    100 non-null    object
 4   text_content  96 non-null     object
dtypes: int64(2), object(3)
memory usage: 4.0+ KB


In [34]:
df['text_content'] = df['text_content'].astype(str)

In [35]:
import re

# Function to clean and extract relevant text
def clean_and_extract(text):
    # Remove special characters, numbers, and extra whitespaces
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()  # Convert text to lowercase for consistency

    # Extract relevant keywords or phrases using regex or specific patterns
    # Example: Extract sentences containing tumor board-related keywords
    relevant_text = []
    sentences = text.split('.')
    keywords = ['tumor board', 'cancer conference', 'multidisciplinary team']
    for sentence in sentences:
        if any(keyword in sentence for keyword in keywords):
            relevant_text.append(sentence.strip())

    return ' '.join(relevant_text)

# Apply the function to clean and extract relevant text for each row in the DataFrame
df['clean_text'] = df['text_content'].apply(clean_and_extract)


In [36]:
df.drop(['text_content'],inplace =True,axis=1)
# train.columns

In [37]:
df.head(1)

,url,doc_id,label,clean_text
0,http://elbe-elster-klinikum.de/fachbereiche/ch...,1,1,
